In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT_ID = os.getenv("LANGCHAIN_PROJECT_ID")

In [7]:
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [8]:
base_Actor="{'name': 'Tom Hanks', film_names': ['Forrest Gump']}"

parser.parse(base_Actor)

OutputParserException: Invalid json output: {'name': 'Tom Hanks', film_names': ['Forrest Gump']}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [9]:
base_Actor

"{'name': 'Tom Hanks', film_names': ['Forrest Gump']}"

In [10]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model='gpt-4o'))

In [11]:
new_parser.get_format_instructions

<bound method OutputFixingParser.get_format_instructions of OutputFixingParser(parser=PydanticOutputParser(pydantic_object=<class '__main__.Actor'>), retry_chain=PromptTemplate(input_variables=['completion', 'error', 'instructions'], input_types={}, partial_variables={}, template='Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F99FC06990>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F99FC0A210>, root_client=<openai.OpenAI object at 0x000001F99F277110>, root_async_client=<openai.AsyncOpenAI object at 0x000001F99FC06AD0>, model_name='gpt-4o', mod

In [12]:
final_Actor=new_parser.parse(base_Actor)

In [13]:
final_Actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])